In [2]:
import pandas as pd

In [ ]:
!pip list

In [3]:
!pip install pyarrow --quiet

In [5]:

train_inputs = pd.read_parquet('train_inputs.parquet')
val_inputs = pd.read_parquet('val_inputs.parquet')

train_targets = pd.read_parquet('train_targets.parquet')
val_targets = pd.read_parquet('val_targets.parquet')

In [ ]:
train_inputs.head()

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Credit_Mix_Good,Credit_Mix_Standard,Payment_of_Min_Amount_No,Payment_of_Min_Amount_Yes,Payment_Behaviour_High_spent_Large_value_payments,Payment_Behaviour_High_spent_Medium_value_payments,Payment_Behaviour_High_spent_Small_value_payments,Payment_Behaviour_Low_spent_Large_value_payments,Payment_Behaviour_Low_spent_Medium_value_payments,Payment_Behaviour_Low_spent_Small_value_payments
75220,0.056534,Journalist,0.003902,0.552466,0.001112,0.002001,0.000173,0.063910,0.263889,0.002045,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
48955,0.057078,Accountant,0.004377,0.629572,0.003335,0.000667,0.001553,0.065163,0.236111,0.002955,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
44966,0.059687,Scientist,0.000872,0.125483,0.003335,0.004670,0.001208,0.064536,0.458333,0.002500,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13568,0.058056,Manager,0.000638,0.104235,0.004447,0.004670,0.001380,0.065163,0.291667,0.003636,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
92727,0.057404,Media_Manager,0.000577,0.110080,0.001668,0.002001,0.001380,0.063283,0.097222,0.002727,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
val_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 75721 to 42410
Data columns (total 46 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Age                                                 20000 non-null  float64
 1   Occupation                                          20000 non-null  object 
 2   Annual_Income                                       20000 non-null  float64
 3   Monthly_Inhand_Salary                               20000 non-null  float64
 4   Num_Bank_Accounts                                   20000 non-null  float64
 5   Num_Credit_Card                                     20000 non-null  float64
 6   Interest_Rate                                       20000 non-null  float64
 7   Num_of_Loan                                         20000 non-null  float64
 8   Delay_from_due_date                                 20000 non-null  float64
 

In [ ]:
test_inputs.shape

(50000, 46)

In [ ]:
train_targets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80000 entries, 75220 to 15795
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Credit_Score  80000 non-null  object
dtypes: object(1)
memory usage: 1.2+ MB


In [ ]:
train_targets['Credit_Score']

75220    Standard
48955        Good
44966        Good
13568    Standard
92727        Good
           ...   
6265         Good
54886        Poor
76820    Standard
860      Standard
15795        Poor
Name: Credit_Score, Length: 80000, dtype: object

In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from  sklearn.ensemble import GradientBoostingClassifier ,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [7]:
def trainModel(Model, metric):
    numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
    X_train = train_inputs[numeric_cols]
    X_val = val_inputs[numeric_cols]

    # Reshape target variables to ensure they are in the correct shape
    y_train = train_targets.values.ravel()
    y_val = val_targets.values.ravel()

    model = Model
    model.fit(X_train, y_train)

    train_preds = model.predict(X_train)
    print(f"Training evaluation: {metric(y_train, train_preds) * 100:.2f}%")

    val_preds = model.predict(X_val)
    print(f"Validation evaluation: {metric(y_val, val_preds) * 100:.2f}%")



In [8]:
#Customized LogisticRegression model for multiclass classification using the 'saga' solver
model = LogisticRegression(
    penalty='l2',          # Regularization technique
    dual=False,            # Dual formulation (only for 'liblinear' solver)
    tol=0.0001,            # Tolerance for stopping criteria
    C=1.0,                 # Inverse of regularization strength
    fit_intercept=True,    # Whether to include intercept
    intercept_scaling=1,   # Scaling of the intercept
    class_weight=None,     # Weights associated with classes
    random_state=None,     # Seed for randomness
    solver='saga',         # Stochastic Average Gradient descent solver
    max_iter=100,          # Maximum number of iterations
    multi_class='multinomial',  # Option for multiclass problems
    verbose=0,             # Verbosity level
    warm_start=False,      # Reuse the solution of the previous call
    n_jobs=None,           # Number of CPU cores used
    l1_ratio=None          # Elastic net mixing parameter (not used here)
)

metric=accuracy_score
trainModel(model,metric)

Training evaluation: 61.81%
Validation evaluation: 61.58%


In [9]:
# Customized DecisionTreeClassifier for multiclass classification with a maximum depth
model = DecisionTreeClassifier(
    criterion='gini',              # Criterion for measuring the quality of a split ('gini' or 'entropy')
    splitter='best',               # Strategy used to choose the split at each node ('best' or 'random')
    max_depth=15,     # Maximum depth of the tree
    min_samples_split=10,           # Minimum number of samples required to split an internal node
    min_samples_leaf=5,
    # Minimum number of samples required to be at a leaf node
    min_weight_fraction_leaf=0.0,  # Minimum weighted fraction of the sum total of weights required to be at a leaf node
    max_features=None,             # Number of features to consider when looking for the best split
    random_state=None,             # Seed used by the random number generator
    max_leaf_nodes=None,           # Grow a tree with max_leaf_nodes in best-first fashion
    min_impurity_decrease=0.0,     # A node will be split if this split induces a decrease of the impurity greater than or equal to this value
    class_weight=None,             # Weights associated with classes
    ccp_alpha=0.0,                 # Complexity parameter used for Minimal Cost-Complexity Pruning            # Constraints on the monotonicity of the features
)
metric=accuracy_score
trainModel(model,metric)


Training evaluation: 79.15%
Validation evaluation: 70.96%


In [10]:
model = GaussianNB(
    priors=None,            # Prior probabilities of the classes
    var_smoothing=1e-9      # Portion of the largest variance of all features added to variances for stability
)
metric=accuracy_score
trainModel(model,metric)

Training evaluation: 57.82%
Validation evaluation: 57.64%


In [11]:
model = GradientBoostingClassifier(
    loss='log_loss',                # Changed loss function to 'log_loss' for multiclass classification
    learning_rate=0.1,              # Kept learning rate the same
    n_estimators=300,               # Reduced number of boosting stages
    subsample=0.8,                  # Reduced subsample
    criterion='friedman_mse',       # Kept criterion the same
    min_samples_split=2,            # Kept min_samples_split the same
    min_samples_leaf=1,             # Kept min_samples_leaf the same
    min_weight_fraction_leaf=0.0,   # Kept min_weight_fraction_leaf the same
    max_depth=3,                    # Reduced max_depth
    min_impurity_decrease=0.0,      # Kept min_impurity_decrease the same
    init=None,                      # Kept init the same
    random_state=42,                # Set random_state for reproducibility
    max_features=None,              # Kept max_features the same
    verbose=0,                      # Kept verbose the same
    max_leaf_nodes=None,            # Kept max_leaf_nodes the same
    warm_start=False,               # Kept warm_start the same
    validation_fraction=0.1,        # Kept validation_fraction the same
    n_iter_no_change=None,          # Kept n_iter_no_change the same
    tol=0.0001,                     # Kept tol the same
    ccp_alpha=0.0                   # Kept ccp_alpha the same
)
metric=accuracy_score
trainModel(model,metric)


Training evaluation: 73.17%
Validation evaluation: 71.34%


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
X_train = train_inputs[numeric_cols]
X_val = val_inputs[numeric_cols]

    # Reshape target variables to ensure they are in the correct shape
y_train = train_targets.values.ravel()
y_val = val_targets.values.ravel()

# Customized GradientBoostingClassifier for multiclass classification
model = GradientBoostingClassifier(
    loss='log_loss',                # Changed loss function to 'log_loss' for multiclass classification
    learning_rate=0.05,             # Reduced learning rate
    n_estimators=300,               # Increased number of boosting stages
    subsample=0.8,                  # Kept subsample the same
    criterion='friedman_mse',       # Kept criterion the same
    min_samples_split=5,            # Increased min_samples_split
    min_samples_leaf=2,             # Increased min_samples_leaf
    min_weight_fraction_leaf=0.0,   # Kept min_weight_fraction_leaf the same
    max_depth=5,                    # Kept max_depth the same
    min_impurity_decrease=0.0,      # Kept min_impurity_decrease the same
    init=None,                      # Kept init the same
    random_state=42,                # Set random_state for reproducibility
    max_features=None,              # Kept max_features the same
    verbose=0,                      # Kept verbose the same
    max_leaf_nodes=None,            # Kept max_leaf_nodes the same
    warm_start=False,               # Kept warm_start the same
    validation_fraction=0.1,        # Kept validation_fraction the same
    n_iter_no_change=None,          # Kept n_iter_no_change the same
    tol=0.0001,                     # Kept tol the same
    ccp_alpha=0.001                 # Increased ccp_alpha for regularization
)

# Perform cross-validation with 5 folds
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

# Print cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Score:", np.mean(cv_scores))


In [12]:
# Customized RandomForestClassifier for multiclass classification
model = RandomForestClassifier(
    n_estimators=500,
    criterion='entropy',
    max_depth=25,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    ccp_alpha=0.001
)
metric=accuracy_score
trainModel(model,metric)

Training evaluation: 68.91%
Validation evaluation: 68.48%


In [13]:
model = KNeighborsClassifier(
    n_neighbors=10,        # Number of neighbors to use by default for k-neighbors queries
    weights='uniform',    # Weight function used in prediction ('uniform' or 'distance')
    algorithm='auto',     # Algorithm used to compute the nearest neighbors ('auto', 'ball_tree', 'kd_tree', 'brute')
    leaf_size=30,         # Leaf size passed to BallTree or KDTree
    p=2,                  # Power parameter for the Minkowski metric
    metric='minkowski',   # Metric to use for distance computation
    n_jobs=None           # The number of parallel jobs to run for neighbors search
)
metric=accuracy_score
trainModel(model,metric)


Training evaluation: 70.10%
Validation evaluation: 64.22%
